In [1]:
import pandas as pd
import numpy as np
import nltk
import requests
import re
import json

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
tab_colors_list = list(mcolors.TABLEAU_COLORS.keys())
import matplotlib.lines as mlines
import seaborn as sns

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

import sddk

In [2]:
# to access gsheet, you need Google Service Account key json file
# I have mine located in my personal space on sciencedata.dk, so I read it from there:
conf = sddk.configure()

# (1) read the file and parse its content
file_data = conf[0].get(conf[1] + "ServiceAccountsKey.json").json()
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)
# (5) establish connection with spreadsheets specified by their url
occupations = gc.open_by_url("https://docs.google.com/spreadsheets/d/1nONTEwp42CVnq3iCiONrFbJedIcYtBV-l4Bil5mU7Eo/edit?usp=sharing")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
endpoint variable has been configured to: https://sciencedata.dk/files/


# Requirements

In [15]:
import json
import re
import pandas as pd
pd.options.display.max_columns = 1000 # to see all columns
import warnings
warnings.filterwarnings('ignore')

# Loading datasets

In [5]:
EDHCS = pd.read_csv("../data/large_data/EDHCSg.csv", low_memory=False, index_col=None)
EDHCS.head(5)

,responsible_individual,type_of_inscription,letter_size,not_after,literature,work_status,height,diplomatic_text,people,depth,material,type_of_monument,province_label,width,transcription,country,uri,findspot_ancient,last_update,modern_region,findspot_modern,language,id,edh_geography_uri,commentary,trismegistos_uri,not_before,external_image_uris,fotos,coordinates,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_conservative,clean_text_interpretive_word,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military,geometry,within_RE,urban_context,within_rome,nearest_city,city_id_hanson,city_pop_est,city_geometry,nearest_city_type,nearest_city_dist,EDCS-ID,publication,province,province_list,place,place_list,end_yr_list,notes_dating,status_list,inscr_type,status_notation,inscr_process,notes_references,notes_comments,inscription,inscription_stripped_final,Links,dating from,dating to,status,Latitude,Longitude,photo,Material,Comment
0,Feraudi,epitaph,4 cm,200.0,"AE 1983, 0080. (A); A. Ferrua, RAL 36, 1981, 1...",no image,28 cm,C SEXTIVS PARIS / QVI VIXIT / ANNIS LXX,"[{'nomen': 'Sextius', 'praenomen': 'C.', 'pers...",NaN,marble: rocks - metamorphic rocks,tabula,Roma,85 cm,C(aius) Sextius Paris / qui vixit / annis LXX,Italy,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Roma,2014-04-07,Lazio,Roma,Latin,HD000002,https://edh-www.adw.uni-heidelberg.de/edh/geog...,AE 1983: Breite: 35 cm.,https://www.trismegistos.org/text/265631,51.0,{},{},"[12.4823, 41.8955]",265631.0,"['http://www.trismegistos.org/place/000172', '...",Caius Sextius Paris qui vixit annis LXX ...,51 AD – 200 AD,21.0,unbestimmt,257.0,Tafel,48.0,Marmor,1000.0,92.0,Grabinschrift,epitaph,Certain,28.0,85.0,NaN,Marble,tabula,Certain,Roma,Certain,Italy,Certain,Roma,Certain,Lazio,Certain,Roma,Certain,"Via Nomentana, S. Alessandro, Kirche",Certain,51 AD – 200 AD,C Sextius Paris qui vixit annis LXX,Caius Sextius Paris qui vixit annis LXX,Caius Sextius Paris qui vixit annis LXX,"Via Nomentana, S. Alessandro, Kirche",1937,NaN,NaN,NaN,NaN,NaN,"[12.4823, 41.8955]",True,big,True,Roma,992,923313.0,"[12.485098, 41.892777]",big,0.003904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Feraudi,honorific inscription,4.5-3 cm,170.0,"AE 1983, 0518. (B); J. González, ZPE 52, 1983,...",provisional,(37) cm,[ ]VMMIO [ ] / [ ]ISENNA[ ] / [ ] XV[ ] / [ ] / [,"[{'nomen': 'Mummius+', 'gender': 'male', 'prae...",(12) cm,marble: rocks - metamorphic rocks,statue base,Baetica,(34) cm,[P(ublio) M]ummio [P(ubli) f(ilio)] / [Gal(eri...,Spain,https://edh-www.adw.uni-heidelberg.de/edh/insc...,NaN,2006-08-31,Sevilla,Tomares,Latin,HD000003,https://edh-www.adw.uni-heidelberg.de/edh/geog...,(B): [S]isenna ist falscher Kasus; folgende E...,https://www.trismegistos.org/text/220675,131.0,{},{},"[-6.0459, 37.3728]",220675.0,"['http://www.trismegistos.org/place/025443', '...",Publio Mummio Publi filio Galeria Sisennae Rut...,131 AD – 170 AD,21.0,unbestimmt,57.0,Statuenbasis,48.0,Marmor,1000.0,69.0,Ehreninschrift,honorific inscription,Certain,37.0,34.0,12.0,Marble,statue base,Certain,Baetica,Certain,Spain,Certain,NaN,NaN,Sevilla,Certain,Tomares,Certain,NaN,NaN,131 AD – 170 AD,ummio isenna Xv,Publio Mummio Publi filio Galeria Sisennae Rut...,Publio Mummio Publi filio Galeria Sisennae Rut...,NaN,before 19

In [6]:
print(EDHCS.columns)

Index(['responsible_individual', 'type_of_inscription', 'letter_size',
       'not_after', 'literature', 'work_status', 'height', 'diplomatic_text',
       'people', 'depth',
       ...
       'inscription_stripped_final', 'Links', 'dating from', 'dating to',
       'status', 'Latitude', 'Longitude', 'photo', 'Material', 'Comment'],
      dtype='object', length=109)


# Function to extract occupations

In [7]:
def extract_occup(inscription_text):
    occups_found = []
    if not isinstance(inscription_text, str): # if not valid string
        inscription_text = ""
    for occup in occups_declined_dict.keys():
        for occup_morph in occups_declined_dict[occup]:
            try:
                if occup_morph in inscription_text: # first check it this way, otherwise skip
                    occup_morph_N = len(re.findall("(\W|^)" + occup_morph + "(\W|$)", inscription_text))
                    occups_found.extend([occup] * occup_morph_N)
                    inscription_text = inscription_text.replace(occup_morph, "")
            except: pass
    return occups_found

# Occupations - extraction

In [8]:
# read declined occupations
with open("../data/occups_declined_dict.json", "r") as fp:
    occups_declined_dict = json.load(fp)

In [9]:
# check that our occupations are properly arranged (from the longest...)
list(occups_declined_dict.keys())[:20]

['exactor auri argenti et aeris',
 'inclusor auri et gemmarum',
 'tesserarius lignarius',
 'refector pectinarius',
 'instructor parietum',
 'tribor argentarius',
 'carrocarpentarius',
 'dactilidiogliphus',
 'manceps salinarum',
 'manceps thermarum',
 'aequator monetae',
 'conchyliolegulus',
 'orchestopolarius',
 'rhypararographus',
 'calciamentarius',
 'chorographarius',
 'diffusor oleari',
 'domnicomontanus',
 'faber tignuarii',
 'medicamentarius']

## Occupations - test on a sample

In [16]:
# sample for testing
EDHCS_sample = EDHCS[60000:65000]
len(EDHCS_sample)

5000

In [17]:
%%time
EDHCS_sample["occups"] = EDHCS_sample["clean_text_interpretive_word"].apply(extract_occup)
EDHCS_occups_list = [el for sublist in EDHCS_sample["occups"].tolist() for el in sublist]
len(EDHCS_occups_list) 
# v1: 3.03s, 946 found
# once we remove after first match, then only 636
# but once we are sensitive to repeated occurences, we get 742
# when looking for within a lower case inscription, the number increases to 902

CPU times: user 2.61 s, sys: 221 µs, total: 2.61 s
Wall time: 2.61 s


103

In [19]:
EDHCS_sample["occups_N"] = EDHCS_sample["occups"].apply(len)

In [21]:
len(EDHCS_sample[EDHCS_sample["occups_N"] > 0])

77

## Occupations - application on the whole dataset

In [21]:
%%time
EDH["occups"] = EDH["clean_text_interpretive_word"].apply(extract_occup)

CPU times: user 44.7 s, sys: 508 µs, total: 44.7 s
Wall time: 44.7 s


In [22]:
EDH_occups_list = [el for sublist in EDH["occups"].tolist() for el in sublist]
print(len(EDH_occups_list))
print(nltk.FreqDist(EDH_occups_list).most_common(30))

2268
[('faber', 400), ('curator', 244), ('sagittarius', 162), ('medicus', 129), ('scriba', 107), ('aerarius', 103), ('vexillarius', 65), ('negotiator', 52), ('mensor', 34), ('dendrophorus', 30), ('gladiator', 30), ('cornicen', 27), ('argentarius', 26), ('arcarius', 26), ('scaenicus', 26), ('architectus', 23), ('conditor', 21), ('nauta', 20), ('negotians', 20), ('tignarius', 17), ('ornatrix', 17), ('lapidarius', 17), ('structor', 15), ('vestiarius', 14), ('navalis', 11), ('armamentarius', 11), ('possessor', 11), ('salarius', 11), ('sutor', 11), ('venator', 11)]


results ignoring both lower and uppercase words (for comparison):

```
2745
[('faber', 407), ('curator', 321), ('medicus', 243), ('scriba', 110), ('aerarius', 107), ('agricola', 89), ('pollio', 81), ('negotiator', 79), ('centonarius', 67), ('vexillarius', 59), ('mercator', 45), ('argentarius', 42), ('mensor', 39), ('cerdo', 36), ('cornicen', 28), ('arcarius', 26), ('conditor', 26), ('architectus', 23), ('nauta', 22), ('dendrophorus', 19), ('tignarius', 18), ('structor', 16), ('ornatrix', 16), ('lapidarius', 16), ('vestiarius', 15), ('sutor', 15), ('venator', 15), ('forensis', 14), ('scutarius', 14), ('coactor', 13)]```

In [23]:
EDH["occups_N"] = EDH["occups"].apply(len)

In [24]:
%%time
EDCS["occups"] = EDCS["clean_text_interpretive_word"].apply(extract_occup)

CPU times: user 3min 19s, sys: 15 ms, total: 3min 19s
Wall time: 3min 19s


In [25]:
EDCS_occups_list = [el for sublist in EDCS["occups"].tolist() for el in sublist]
print(len(EDCS_occups_list))
print(nltk.FreqDist(EDCS_occups_list).most_common(30))

7685
[('faber', 1116), ('curator', 735), ('aerarius', 458), ('medicus', 435), ('scriba', 389), ('sagittarius', 339), ('argentarius', 139), ('mensor', 128), ('dendrophorus', 119), ('vexillarius', 108), ('figulus', 97), ('negotiator', 97), ('gladiator', 92), ('vestiarius', 90), ('scaenicus', 83), ('architectus', 80), ('arcarius', 76), ('conditor', 75), ('ornatrix', 61), ('negotians', 60), ('structor', 55), ('nauta', 51), ('pistor', 50), ('vinarius', 49), ('fullo', 48), ('marmorarius', 47), ('cocus', 44), ('nummularius', 43), ('lecticarius', 42), ('navalis', 41)]


previous version
```
10085
[('faber', 1152), ('curator', 915), ('medicus', 668), ('aerarius', 513), ('scriba', 392), ('cocus', 289), ('vexillarius', 260), ('cerdo', 254), ('pollio', 245), ('agricola', 225), ('mercator', 217), ('centonarius', 217), ('argentarius', 170), ('negotiator', 159), ('figulus', 138), ('mensor', 136), ('vestiarius', 104), ('dendrophorus', 91), ('lanius', 89), ('conditor', 89), ('copo', 86), ('architectus', 81), ('arcarius', 79), ('nauta', 68), ('vinarius', 59), ('structor', 58), ('ornatrix', 56), ('fullo', 52), ('pistor', 52), ('sutor', 48)]
```

In [26]:
EDCS["occups_N"] = EDCS["occups"].apply(len)

## Occupations - overview

In [27]:
print("EDH - number of occupation occurances: " + str(EDH["occups_N"].sum()))
print("EDH - number of inscriptions with at least one occupation mentioned: " + str(len(EDH[EDH["occups_N"] > 0])))
print("EDH - number of dated inscriptions with at least one occupation mentioned: "+ str(len(EDH[(EDH["not_before"].notnull()) & (EDH["occups_N"] > 0)])))
print("EDH - number of occupation occurances in dated inscriptions: " + str(EDH[EDH["not_before"].notnull()]["occups_N"].sum()))

EDH - number of occupation occurances: 2268
EDH - number of inscriptions with at least one occupation mentioned: 1913
EDH - number of dated inscriptions with at least one occupation mentioned: 1224
EDH - number of occupation occurances in dated inscriptions: 1450


In [28]:
print("EDCS - number of occupation occurances: " + str(EDCS["occups_N"].sum()))
print("EDCS - number of inscriptions with at least one occupation mentioned: " + str(len(EDCS[EDCS["occups_N"] > 0])))
print("EDCS - number of dated inscriptions with at least one occupation mentioned: "+ str(len(EDCS[(EDCS["dating to"].notnull()) & (EDCS["occups_N"] > 0)])))
print("EDCS - number of occupation occurances in dated inscriptions: " + str(EDCS[EDCS["dating to"].notnull()]["occups_N"].sum()))

EDCS - number of occupation occurances: 7685
EDCS - number of inscriptions with at least one occupation mentioned: 6371
EDCS - number of dated inscriptions with at least one occupation mentioned: 4008
EDCS - number of occupation occurances in dated inscriptions: 4879


# Organizations - extraction

In [30]:
# read declined occupations
with open("../data/organizations_declined_dict.json", "r") as fp:
    occups_declined_dict = json.load(fp)

In [31]:
# check that our occupations are properly arranged (from the longest...)
list(occups_declined_dict.keys())[:15]

['collegatarius',
 'collegiarius',
 'collegiatus',
 'corporatus',
 'sodalicium',
 'collegium',
 'collegius',
 'concilium',
 'conlegium',
 'sodalitas',
 'colegium',
 'sociatio',
 'societas',
 'collega',
 'corpus']

## Organizations - test on a sample data

In [32]:
# sample for testing
EDH_sample = EDH[:1000]
len(EDH_sample)

1000

In [33]:
%%time
EDH_sample["organizations"] = EDH_sample["clean_text_interpretive_word"].apply(extract_occup)
EDH_organizations_list = [el for sublist in EDH_sample["organizations"].tolist() for el in sublist]
len(EDH_organizations_list) 
# v1: 3.03s, 946 found
# once we remove after first match, then only 636
# but once we are sensitive to repeated occurences, we get 742
# when looking for within a lower case inscription, the number increases to 902

CPU times: user 28.8 ms, sys: 12 µs, total: 28.8 ms
Wall time: 28 ms


/home/kasev/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



49

In [34]:
EDH_sample["organizations_N"] = EDH_sample["organizations"].apply(len)

/home/kasev/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [35]:
len(EDH_sample[EDH_sample["organizations_N"] > 0])

18

In [37]:
# create sample and export it to gsheet
EDH_occups_sample = EDH_sample[EDH_sample["organizations_N"] > 0][["id", "clean_text_interpretive_word", "organizations", "type_of_inscription_clean"]]
set_with_dataframe(occupations.add_worksheet("EDH_orgs_sample", 1, 1), EDH_occups_sample)

## Organizations - application to full datasets

In [38]:
%%time
EDH["organizations"] = EDH["clean_text_interpretive_word"].apply(extract_occup)

CPU times: user 1.19 s, sys: 34 µs, total: 1.19 s
Wall time: 1.18 s


In [39]:
EDH_organizations_list = [el for sublist in EDH["organizations"].tolist() for el in sublist]
print(len(EDH_organizations_list))
print(nltk.FreqDist(EDH_organizations_list).most_common(30))

868
[('collegium', 452), ('corpus', 288), ('collega', 60), ('conlegium', 13), ('concilium', 12), ('corporatus', 11), ('collegiatus', 11), ('collegius', 10), ('societas', 5), ('sodalicium', 4), ('collegiarius', 1), ('colegium', 1)]


In [40]:
EDH["organizations_N"] = EDH["organizations"].apply(len)

In [41]:
%%time
EDCS["organizations"] = EDCS["clean_text_interpretive_word"].apply(extract_occup)

CPU times: user 4.26 s, sys: 16 ms, total: 4.28 s
Wall time: 4.28 s


In [42]:
EDCS_organizations_list = [el for sublist in EDCS["organizations"].tolist() for el in sublist]
print(len(EDCS_organizations_list))
print(nltk.FreqDist(EDCS_organizations_list).most_common(30))

2974
[('collegium', 1558), ('corpus', 954), ('collega', 178), ('colegium', 74), ('corporatus', 71), ('concilium', 40), ('collegius', 34), ('sodalicium', 24), ('collegiatus', 18), ('societas', 11), ('conlegium', 9), ('sodalitas', 2), ('collegiarius', 1)]


In [43]:
EDCS["organizations_N"] = EDCS["organizations"].apply(len)

## Organizations - overview

In [44]:
print("EDH - number of organization occurances: " + str(EDH["organizations_N"].sum()))
print("EDH - number of inscriptions with at least one organization mentioned: " + str(len(EDH[EDH["organizations_N"] > 0])))
print("EDH - number of dated inscriptions with at least one organization mentioned: "+ str(len(EDH[(EDH["not_before"].notnull()) & (EDH["organizations_N"] > 0)])))
print("EDH - number of organization occurances in dated inscriptions: " + str(EDH[EDH["not_before"].notnull()]["organizations_N"].sum()))

EDH - number of organization occurances: 868
EDH - number of inscriptions with at least one organization mentioned: 727
EDH - number of dated inscriptions with at least one organization mentioned: 529
EDH - number of organization occurances in dated inscriptions: 631


In [45]:
print("EDCS - number of organizations occurances: " + str(EDCS["organizations_N"].sum()))
print("EDCS - number of inscriptions with at least one organizations mentioned: " + str(len(EDCS[EDCS["organizations_N"] > 0])))
print("EDCS - number of dated inscriptions with at least one organizations mentioned: "+ str(len(EDCS[(EDCS["dating to"].notnull()) & (EDCS["organizations_N"] > 0)])))
print("EDCS - number of organizations occurances in dated inscriptions: " + str(EDCS[EDCS["dating to"].notnull()]["organizations_N"].sum()))

EDCS - number of organizations occurances: 2974
EDCS - number of inscriptions with at least one organizations mentioned: 2223
EDCS - number of dated inscriptions with at least one organizations mentioned: 1509
EDCS - number of organizations occurances in dated inscriptions: 2143


## Saving to Sciencedata

In [46]:
# login to our project folder, owned by my AU account 648597@au.dk
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [47]:
sddk.write_file("SDAM_data/social_diversity/EDH_occupsorgs_2021-02-26.json", EDH, conf)
sddk.write_file("SDAM_data/social_diversity/EDCS_occupsorgs_2021-02-26.json", EDCS, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/social_diversity/EDH_occupsorgs_2021-02-26.json"
Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/social_diversity/EDCS_occupsorgs_2021-02-26.json"
